# Import requried dependencies

In [14]:
import os
import pandas as pd
import numpy as np

# Defining Local paths to datasets

In [2]:
datasets_path = "../../../datasets"
auxillary_path = f"{datasets_path}/auxiliary-data"
transformed_path = f"{datasets_path}/transformed"

coe_inp_path = f"{auxillary_path}/sg-coe-prices.csv"
stock_inp_path = f"{auxillary_path}/sg-stock-prices.csv"

# Other Constants which are needed

In [3]:
month_to_num_map = {
    "january": "01",
    "february": "02",
    "march": "03",
    "april": "04",
    "may": "05",
    "june": "06",
    "july": "07",
    "august": "08",
    "september": "09",
    "october": "10",
    "november": "11",
    "december": "12",
}


# 1. Analysing the COE Auxilary dataset

<b><u> What is the dataset about mainly </u></b>

In the context of Singapore cars, "COE" stands for "Certificate of Entitlement." It is a unique and significant component of Singapore's vehicle ownership and registration system. The COE is essentially a license that grants the holder the right to register, own, and use a vehicle in Singapore for a specific period, usually ten years.

There is usually a bidding process where a car of a particular category is usually auctioned off.

The COE system is implemented in Singapore as a measure to regulate and control the number of vehicles on the road due to limited land resources and congestion issues. It also serves as a revenue source for the government.

<b><u> Explanation of each field </u></b>

1. year-month : year and month when a particular car was auctioned
2. category: category of car (e.g. luxury, sport, trucks etc). There is some ordinality here but we are not considering it since we will look at the price instead
3. bidding: the bidding round when the car was auctioned
4. quota: what is the quota of no of bids accepted
5. bids: No of bids people raised in the interest of buying a particular car
6. price: the final price the car got sold for

<b><u> What can we extract from this? </u></b>

With this dataset we can indirectly infer the current economic conditions as the more no of people who are ready to buy a car indicates how "well off" people are financially to even consider such an option.

Based on the ratio of the number of bids to the existing quota it shows the "interest" of how contested/popular a particular car is

<b><u> What is the output we are extracting </u></b>

We will come up with a "coe-indicator" score which is a number which represents the state of the economy for a particular month. How we come up with this number will be explained below

In [4]:
# read the input df
coe_prices_df = pd.read_csv(coe_inp_path)

#transform the month to a number based on the map defined earlier
coe_prices_df["month"] = coe_prices_df["month"].map(month_to_num_map)

# create a new column called year which is of format YYYY-mm
coe_prices_df["date"] = pd.to_datetime(coe_prices_df['year'].astype(str) + '-' + coe_prices_df['month'].astype(str), format="%Y-%m").dt.strftime('%Y-%m')



## Creating a price indicator

Here as you can see we essentially do

(bids/quota) * price

which means out of the existing quota, how many bids came for a particular car category multiplied with the price.

This ratio can be both >1 or <1 as that is an indication of how "interested" people are to buy this particular car.

We find this indicator and then do min-max normalization

In [5]:
# getting price indicator for each row
coe_prices_df["coe_price_indicator"] = (coe_prices_df["bids"] / coe_prices_df["quota"]) * coe_prices_df["price"]

# normalize the price indicator values between 0-1
coe_prices_price_indicator_min = coe_prices_df["coe_price_indicator"].min()
coe_prices_price_indicator_max = coe_prices_df["coe_price_indicator"].max()
coe_prices_df["coe_price_indicator"] = (coe_prices_df["coe_price_indicator"] - coe_prices_price_indicator_min) / (
        coe_prices_price_indicator_max - coe_prices_price_indicator_min)

## Finding cummulative month wise statistics

1. We first perform grouping across date & category and taking the mean of the coe-indicator.
2. Using this df , we find out the total coe_price across all categories.

Eventually this will give us some number which is indicative of the economic condition for every month across all categories of cars

In [6]:
# grouping by year and category
avg_price_indicator_per_month_per_category = coe_prices_df.groupby(["date", "category"])[
    "coe_price_indicator"].mean().reset_index()
total_price_indicator_per_month = avg_price_indicator_per_month_per_category.groupby(["date"])[
    "coe_price_indicator"].sum().reset_index()

In [7]:
total_price_indicator_per_month.head(30)

,date,coe_price_indicator
0,2021-01,0.251816
1,2021-02,0.208422
2,2021-03,0.185832
3,2021-04,0.572866
4,2021-05,0.424839
5,2021-06,0.454947
6,2021-07,0.425407
7,2021-08,0.476939
8,2021-09,0.559383
9,2021-10,0.849635


## What to do about the missing months?

Since there are few months which dont have any coe-indicator value, we basically do imputation by finding the mean for that year and then assign those values

In [8]:
# Create a date range with all months from the minimum to maximum date
start_date = pd.to_datetime('2021-01', format='%Y-%m')
end_date = pd.to_datetime('2023-12', format='%Y-%m')
date_range = pd.date_range(start=start_date, end=end_date, freq='M')
date_df = pd.DataFrame({'date': date_range})
date_df["date"] = date_df["date"].dt.strftime('%Y-%m')

# Merge date_df with total_price_indicator_per_month to fill missing months
merged_df = date_df.merge(total_price_indicator_per_month, on='date', how='left')

# adding month and the year as separate columns
merged_df[["year", "month"]] = merged_df["date"].str.split("-", expand=True)
total_price_indicator_per_month[["year", "month"]] = total_price_indicator_per_month["date"].str.split("-", expand=True)

# Fill missing prices with the calculated average
average_price_by_year = total_price_indicator_per_month.groupby(["year"])['coe_price_indicator'].mean()

# Apply the fill_missing_with_average function to fill missing values
merged_df['coe_price_indicator'] = merged_df.apply(
    lambda row: average_price_by_year.get(row["year"], 0) if pd.isna(row['coe_price_indicator']) else row['coe_price_indicator']
    , axis=1
)

# drop the year and month from this
merged_df = merged_df.drop(columns=["year", "month"])

# we will now have the coe_price_indicator for all months across all the years shown in the test dataset. (which is 2021 -> 2023)

In [9]:
merged_df.head(20)

,date,coe_price_indicator
0,2021-01,0.251816
1,2021-02,0.208422
2,2021-03,0.185832
3,2021-04,0.572866
4,2021-05,0.424839
5,2021-06,0.454947
6,2021-07,0.425407
7,2021-08,0.476939
8,2021-09,0.559383
9,2021-10,0.849635


# 2. Analysing the Stock Auxiliary dataset

<b><u> What is the dataset about mainly </u></b>

This dataset basically has the trading information for different tickers on different dates along with its opening & closing values

<b><u> Explanation of each field </u></b>

1. name: Company name
2. symbol: Company ticket
3. date: exact date
4. open: opening price
5. high: highest price in a day
6. low: lowest price in a day
7. close: closing price
8. adjusted_close: some financial metric applied on the closing value ( not particularly important what exactly it is)

<b><u> What can we extract from this? </u></b>

Since we have daily trading prices for each company, we can first find out the monthly average trading price for each company.

Using this we can take an average of the trading values across all companies to get one "score" which is a number indicating how well all companies performed in a particular month.

This score is indirectly indicative of how the economy is performing which will help in house price prediction.


<b><u> What is the output we are extracting </u></b>

Similar to the coe-indicator we will come up with a stock-indicator which represents the state of the economy. The final output will also basically have month, stock_indicator_score value

In [10]:
# SOME BASIC PREPROCESSING

stock_prices_df = pd.read_csv(stock_inp_path)

# we only care about the adjusted close price
stock_prices_df["stock_price"] = stock_prices_df["adjusted_close"]
stock_prices_df["date"] = pd.to_datetime(stock_prices_df["date"])

# Filter rows where the year is greater than or equal to 2021 ( because that is what the testing data has)
stock_prices_df = stock_prices_df[stock_prices_df['date'].dt.year >= 2021]

# Extract year and month into a new column 'year_month'
stock_prices_df['date'] = stock_prices_df['date'].dt.strftime('%Y-%m')

# drop unnecessary columns
stock_prices_df = stock_prices_df.drop(columns=["symbol", "open", "high", "low", "close", "adjusted_close"])

# normalize the price indicator values between 0-1
stock_price_min = stock_prices_df["stock_price"].min()
stock_price_max = stock_prices_df["stock_price"].max()
stock_prices_df["stock_price"] = (stock_prices_df["stock_price"] - stock_price_min) / (stock_price_max - stock_price_min)


Grouping by company name and date to find the average monthly closing prices for all companies.

Using this we take a mean by grouping on month to find out the "economic" state for a particular month across all companies

In [11]:
# Group by 'name' and 'year_month' to calculate the total stock price for each month for each company
stock_prices_for_company_for_month_df = stock_prices_df.groupby(['name', 'date'])['stock_price'].sum().reset_index()

# Group by 'date' and find the average stock price across companies for each particular month
avg_stock_price_per_month = stock_prices_for_company_for_month_df.groupby(["date"])["stock_price"].mean().reset_index()

Similar to previous dataset, for missing months we just impute using the average for that year

In [12]:
# Create a date range with all months from the minimum to maximum date
start_date = pd.to_datetime('2021-01', format='%Y-%m')
end_date = pd.to_datetime('2024-01', format='%Y-%m')
date_range = pd.date_range(start=start_date, end=end_date, freq='M')
date_df = pd.DataFrame({'date': date_range})
date_df["date"] = date_df["date"].dt.strftime('%Y-%m')

# Merge date_df with total_price_indicator_per_month to fill missing months
merged_df = date_df.merge(avg_stock_price_per_month, on='date', how='left')

# adding month and the year as separate columns
merged_df[["year", "month"]] = merged_df["date"].str.split("-", expand=True)
avg_stock_price_per_month[["year", "month"]] = avg_stock_price_per_month["date"].str.split("-", expand=True)

# Fill missing prices with the calculated average
average_price_by_year = avg_stock_price_per_month.groupby(["year"])['stock_price'].mean()

# Apply the fill_missing_with_average function to fill missing values
merged_df['stock_price'] = merged_df.apply(
    lambda row: average_price_by_year.get(row["year"], 0) if pd.isna(row['stock_price']) else row['stock_price']
    , axis=1
)

# drop the year and month from this
merged_df = merged_df.drop(columns=["year", "month"])

In [13]:
merged_df.head()

,date,stock_price
0,2021-01,0.665302
1,2021-02,0.713294
2,2021-03,0.830401
3,2021-04,0.801463
4,2021-05,0.727715


# 3. How are we going to use these in our project?

Since we have the month and its corresponding indicator values ( both coe & stock) we will use these values and add 2 new columns in the original dataset where we use each row's particular month to fill the value indicator values